<a href="https://colab.research.google.com/github/gc2321/3546-Deep-Learning/blob/main/pytorch/4_1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [42]:
file = "/content/gdrive/MyDrive/neural_data/shakespeare.txt"

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
with open(file,'r',encoding='utf8') as f:
    text = f.read()

In [45]:
text[:1000]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee.\n\n\n                     2\n  When forty winters shall besiege thy brow,\n  And dig deep trenches in thy beauty's field,\n  Thy youth's proud livery so gazed on now,\n  Will be a tattered weed of small worth held:  \n  Then being asked, where all thy beauty lies,\n  Where all the treasure of thy lusty days;\n  To sa

In [46]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [47]:
len(text)

5445609

## Encode Entire Text

In [48]:
all_characters = set(text)

In [49]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [50]:
decoder = dict(enumerate(all_characters))

In [51]:
encoder = {char: ind for ind,char in decoder.items()}

In [52]:
encoded_text = np.array([encoder[char] for char in text])

In [53]:
encoded_text[:500]

array([ 6, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 46,  6, 20, 20, 57, 53, 73, 76, 20,  0, 50, 54,
       53, 18, 17, 16, 20, 45, 53, 18, 50, 16, 19, 53, 18, 17, 20, 14, 18,
       20, 41, 18, 17, 54, 53, 18, 20, 54, 74, 45, 53, 18, 50, 17, 18, 38,
        6, 20, 20, 65, 33, 50, 16, 20, 16, 33, 18, 53, 18, 77, 61, 20, 77,
       18, 50, 19, 16, 61, 28, 17, 20, 53, 73, 17, 18, 20, 76, 54,  5, 33,
       16, 20, 74, 18, 22, 18, 53, 20, 41, 54, 18, 38,  6, 20, 20, 35, 19,
       16, 20, 50, 17, 20, 16, 33, 18, 20, 53, 54, 69, 18, 53, 20, 17, 33,
       73, 19, 15, 41, 20, 77, 61, 20, 16, 54, 76, 18, 20, 41, 18, 45, 18,
       50, 17, 18, 38,  6, 20, 20, 26, 54, 17, 20, 16, 18, 74, 41, 18, 53,
       20, 33, 18, 54, 53, 20, 76, 54,  5, 33, 16, 20, 77, 18, 50, 53, 20,
       33, 54, 17, 20, 76, 18, 76, 73, 53, 61, 48,  6, 20, 20, 35, 19, 16,
       20, 16, 33, 73, 19, 20, 45, 73, 74, 16, 53, 50, 45, 16, 18, 41, 20,
       16, 73, 20, 16, 33

## One Hot Encoding

In [54]:
def one_hot_encoder(encoded_text, num_uni_chars):
    '''
    encoded_text : batch of encoded text

    num_uni_chars = number of unique characters (len(set(text)))
    '''

    # METHOD FROM:
    # https://stackoverflow.com/questions/29831489/convert-encoded_textay-of-indices-to-1-hot-encoded-numpy-encoded_textay

    # Create a placeholder for zeros.
    one_hot = np.zeros((encoded_text.size, num_uni_chars))

    # Convert data type for later use with pytorch (errors if we dont!)
    one_hot = one_hot.astype(np.float32)

    # Using fancy indexing fill in the 1s at the correct index locations
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0


    # Reshape it so it matches the batch sahe
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))

    return one_hot

In [55]:
one_hot_encoder(np.array([1,2,0]),3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

## Creating Training Batches

In [56]:
example_text = np.arange(10)

In [57]:
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [58]:
# If we wanted 5 batches
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [59]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):

    '''
    Generate (using yield) batches for training.

    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one

    Example:

    X:

    [[1 2 3]]

    Y:

    [[ 2 3 4]]

    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence

    '''

    # Total number of characters per batch
    # Example: If samp_per_batch is 2 and seq_len is 50, then 100
    # characters come out per batch.
    char_per_batch = samp_per_batch * seq_len


    # Number of batches available to make
    # Use int() to roun to nearest integer
    num_batches_avail = int(len(encoded_text)/char_per_batch)

    # Cut off end of encoded_text that
    # won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]


    # Reshape text into rows the size of a batch
    encoded_text = encoded_text.reshape((samp_per_batch, -1))


    # Go through each row in array.
    for n in range(0, encoded_text.shape[1], seq_len):

        # Grab feature characters
        x = encoded_text[:, n:n+seq_len]

        # y is the target shifted over by 1
        y = np.zeros_like(x)

        #
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1]  = encoded_text[:, n+seq_len]

        # FOR POTENTIAL INDEXING ERROR AT THE END
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]

        yield x, y

### Example of generating a batch

In [60]:
sample_text = encoded_text[:20]

In [61]:
sample_text

array([ 6, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20])

In [62]:
batch_generator = generate_batches(sample_text,samp_per_batch=2,seq_len=5)

In [63]:
x, y = next(batch_generator)

In [64]:
x

array([[ 6, 20, 20, 20, 20],
       [20, 20, 20, 20, 20]])

In [65]:
y

array([[20, 20, 20, 20, 20],
       [20, 20, 20, 20, 20]])

### GPU Check

In [66]:
torch.cuda.is_available()

True

## Creating the LSTM Model

In [67]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5,use_gpu=False):

        # SET UP ATTRIBUTES
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        #CHARACTER SET, ENCODER, and DECODER
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind,char in decoder.items()}

        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

    def forward(self, x, hidden):

        lstm_output, hidden = self.lstm(x, hidden)
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        final_out = self.fc_linear(drop_output)
        return final_out, hidden

    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''

        if self.use_gpu:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))

        return hidden

### Instantiate the Model

In [68]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=True,
)

In [69]:
total_param  = []
for p in model.parameters():
    total_param.append(int(p.numel()))

In [70]:
sum(total_param)

5470292

In [71]:
len(encoded_text)

5445609

### Optimizer and Loss

In [72]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

## Training Data and Validation Data

In [73]:
# percentage of data to be used for training
train_percent = 0.1

In [74]:
len(encoded_text)

5445609

In [75]:
int(len(encoded_text) * (train_percent))

544560

In [76]:
train_ind = int(len(encoded_text) * (train_percent))

In [77]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

## Training the Network

In [78]:
## VARIABLES

# Epochs to train for
epochs = 50
# batch size
batch_size = 128

# Length of sequence
seq_len = 100

# for printing report purposes
# always start at 0
tracker = 0

# number of characters in text
num_char = max(encoded_text)+1

In [79]:
# Set model to train
model.train()

# Check to see if using GPU
if model.use_gpu:
    model.cuda()

for i in range(epochs):

    hidden = model.hidden_state(batch_size)


    for x,y in generate_batches(train_data,batch_size,seq_len):

        tracker += 1

        # One Hot Encode incoming data
        x = one_hot_encoder(x,num_char)

        # Convert Numpy Arrays to Tensor

        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        # Adjust for GPU if necessary

        if model.use_gpu:

            inputs = inputs.cuda()
            targets = targets.cuda()

        # Reset Hidden State
        # If we dont' reset we would backpropagate through all training history
        hidden = tuple([state.data for state in hidden])

        model.zero_grad()

        lstm_output, hidden = model.forward(inputs,hidden)
        loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())

        loss.backward()

        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
        nn.utils.clip_grad_norm_(model.parameters(),max_norm=5)

        optimizer.step()

        ###################################
        ### CHECK ON VALIDATION SET ######
        #################################

        if tracker % 25 == 0:

            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()

            for x,y in generate_batches(val_data,batch_size,seq_len):

                # One Hot Encode incoming data
                x = one_hot_encoder(x,num_char)


                # Convert Numpy Arrays to Tensor

                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                # Adjust for GPU if necessary

                if model.use_gpu:

                    inputs = inputs.cuda()
                    targets = targets.cuda()

                # Reset Hidden State
                # If we dont' reset we would backpropagate through
                # all training history
                val_hidden = tuple([state.data for state in val_hidden])

                lstm_output, val_hidden = model.forward(inputs,val_hidden)
                val_loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())

                val_losses.append(val_loss.item())

            # Reset to training model after val for loop
            model.train()

            print(f"Epoch: {i} Step: {tracker} Val Loss: {val_loss.item()}")

Epoch: 0 Step: 25 Val Loss: 3.237412214279175
Epoch: 1 Step: 50 Val Loss: 3.2338836193084717
Epoch: 1 Step: 75 Val Loss: 3.2369515895843506
Epoch: 2 Step: 100 Val Loss: 3.225144386291504
Epoch: 2 Step: 125 Val Loss: 3.0921459197998047
Epoch: 3 Step: 150 Val Loss: 2.996035099029541
Epoch: 4 Step: 175 Val Loss: 2.85251522064209
Epoch: 4 Step: 200 Val Loss: 2.725895643234253
Epoch: 5 Step: 225 Val Loss: 2.6071765422821045
Epoch: 5 Step: 250 Val Loss: 2.4977054595947266
Epoch: 6 Step: 275 Val Loss: 2.380185127258301
Epoch: 7 Step: 300 Val Loss: 2.3210794925689697
Epoch: 7 Step: 325 Val Loss: 2.267202615737915
Epoch: 8 Step: 350 Val Loss: 2.2267050743103027
Epoch: 8 Step: 375 Val Loss: 2.1830482482910156
Epoch: 9 Step: 400 Val Loss: 2.151986598968506
Epoch: 10 Step: 425 Val Loss: 2.126530408859253
Epoch: 10 Step: 450 Val Loss: 2.1008572578430176
Epoch: 11 Step: 475 Val Loss: 2.074312686920166
Epoch: 11 Step: 500 Val Loss: 2.0518405437469482
Epoch: 12 Step: 525 Val Loss: 2.0314815044403076
E

## Save Model

In [79]:
# model_name = 'example.net'
# torch.save(model.state_dict(),model_name)

## Load Model

In [80]:
# model = CharModel(
#     all_chars=all_characters,
#     num_hidden=512,
#     num_layers=3,
#     drop_prob=0.5,
#     use_gpu=True,
# )

# model.load_state_dict(torch.load(model_name))
# model.eval()

## Generate Predictions

In [81]:
def predict_next_char(model, char, hidden=None, k=1):

        # Encode raw letters with model
        encoded_text = model.encoder[char]

        # set as numpy array for one hot encoding
        # NOTE THE [[ ]] dimensions!!
        encoded_text = np.array([[encoded_text]])

        # One hot encoding
        encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))

        # Convert to Tensor
        inputs = torch.from_numpy(encoded_text)

        # Check for CPU
        if(model.use_gpu):
            inputs = inputs.cuda()


        # Grab hidden states
        hidden = tuple([state.data for state in hidden])


        # Run model and get predicted output
        lstm_out, hidden = model(inputs, hidden)


        # Convert lstm_out to probabilities
        probs = F.softmax(lstm_out, dim=1).data



        if(model.use_gpu):
            # move back to CPU to use with numpy
            probs = probs.cpu()


        # k determines how many characters to consider
        # for our probability choice.
        # https://pytorch.org/docs/stable/torch.html#torch.topk

        # Return k largest probabilities in tensor
        probs, index_positions = probs.topk(k)


        index_positions = index_positions.numpy().squeeze()

        # Create array of probabilities
        probs = probs.numpy().flatten()

        # Convert to probabilities per index
        probs = probs/probs.sum()

        # randomly choose a character based on probabilities
        char = np.random.choice(index_positions, p=probs)

        # return the encoded value of the predicted char and the hidden state
        return model.decoder[char], hidden

In [82]:
def generate_text(model, size, seed='The', k=1):



    # CHECK FOR GPU
    if(model.use_gpu):
        model.cuda()
    else:
        model.cpu()

    # Evaluation mode
    model.eval()

    # begin output from initial seed
    output_chars = [c for c in seed]

    # intiate hidden state
    hidden = model.hidden_state(1)

    # predict the next character for every character in seed
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)

    # add initial characters to output
    output_chars.append(char)

    # Now generate for size requested
    for i in range(size):

        # predict based off very last letter in output_chars
        char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)

        # add predicted character
        output_chars.append(char)

    # return string of predicted text
    return ''.join(output_chars)

In [83]:
print(generate_text(model, 1000, seed='The ', k=3))

The vvvZ)ZZ)ZZZ))vvvvZ)))vvZZZZZ)vv)ZZ)Z)Z)vZ)Z)Zv)))ZZ))Z)vvZZ)ZZvvv)v))vZ)v)vvZvZ))vZZv))Z)v)Z)v)))Z)))v))vZZv)vZv)))ZZvZZZ))Z)vZvZZ))ZvZZvv)vZZ))v)ZZ)ZZZ)ZZ))ZZvvvv)))Z))vv)v))vZZZv)vZZvZ)ZZZ)vZ)Zv)vZvZ))Z)v))v)ZvvvZ)vvZvZv)ZZ))v)v)))vZZZZv)ZZvZ)v)ZZ)vZZZvvZ)))Zv)Z)ZZvvZZvZZ)ZvZ))Zvv)ZZ)vZ)))vZvZvZv)v)))))))vZvvZZ))vZv)Zvv)v)vZv)vZvvZ)ZvZZv)vZvvZ)vvZv)vZ)))Z)ZvZv))vv)ZvvZvZvv)Z))vvZvZvv))))ZZ)Z))Z)))ZZZ)Z))v)vv)Zvv)Z)vvZ))Z)))ZZ)v)v)ZvZvZZ)vZ))vvvvZ)vZ)Zv)v)ZZ))vZ)vvvZZvv)vvZ)ZZZZ)))Z))))ZZZZ))ZZ)Z)vZ)ZZv)ZvZZvvvvvv)ZZZv)vv))v)ZZZZvZvZZZvv)ZZv)vZZvZZZZvZZ)Zvv)))ZvZZZvZZ)vv)ZZ)v))ZZZZZ))Z))vZ)v))v)vZv))Zv)v))ZZvZ)v))vZv)v)))ZZZZvv)Z))vZ)Z)ZvZZZv)Zv))vvvZZvZZ))ZvZvv)vZZ)Z)Zv)vZv)vvvv)))Z)Z)Z)ZZ))vZZ))ZZZ)v)Zv)ZZvZ))vZZvvZZZZ)Z)Z)ZZ)ZZ)ZvZZv)Z)ZvZZvvZv)v)Zv)vvv)ZZ)ZZ)v)ZZ))vv)vZ))v)vv)vZ)vZv)vZ)Z)vZvv)Z)vZ))ZvZZvZvZ))Z)))v)v))ZvZ)ZZ)vZ)ZZvv)vZ)vZZ)vvZ)ZZZ)v)))v)))vvv))v)Z))vvZ)v)ZZv)vZ)Zvvv)vZ)v))Zv)vZZvvZZZvvv)Z)Zvvvv)vvZvv)ZvZvv)vvvZZvv)vv)ZZZZZZ))v)Z)Z))v)))Z)Zvv)Z)Z)))ZZ)vvv)vZZ))Z